In [1]:
#===============================================================================
# 12_more_FB_data.ipynb
# Purpose: collect and preprocess more data from Facebook Marketing API to test divergent validity.
# Author: Yuanmo He
#===============================================================================

from pysocialwatcher import watcherAPI
import pandas as pd
import numpy as np
import json
import time

# We use the library pysocialwatcher to query Facebook Marketing watcherAPI
# details about the library setup https://github.com/joaopalotti/pySocialWatcher
# more guidance https://worldbank.github.io/connectivity_mapping/intro.html

# Data collection

## get the brands' interest ids

In [6]:
# FB_credentials.csv is where Facebook credentials are saved.
watcher = watcherAPI(api_version="15.0")
watcher.load_credentials_file("FB_credentials.csv")

In [ ]:
# collect the Facebook interest ids for the brands
brands_on_twitter = pd.read_csv("../brands_on_twitter.csv")
brands = brands_on_twitter['brand']
rowlst = []
errorlst = []

for i in range(len(brands)):
    
    try:
        rowlst.append(dict(watcher.get_interests_given_query(brands[i]).loc[0]))
        
    except Exception as e:
        errorlst.append({i: e})
    print(i)
    
print(len(rowlst)) # 315 brands founded as of November 2022
print(len(errorlst)) # 26 errors, all KeyError, so need manual check

In [3]:
print(len(rowlst)) # 315 brands founded as of November 2022
print(len(errorlst)) # 26 errors, all KeyError, so need manual check

341

In [ ]:
# save interest id and manual check 
FB_interest_id = pd.DataFrame(rowlst)
error_ids = [list(i.keys())[0] for i in errorlst]
FB_interest_id.insert(0, 'brand', brands.drop(error_ids).reset_index(drop = True))
FB_interest_id = FB_interest_id.rename(columns = {'name': "FB_name"})
FB_interest_id.to_excel('FB_interest_id_raw.xlsx', index = False) 

# saving to excel because I need to manually add the other brands, and alter werid ones
# label the weird ones in FB_interest_id_raw.xlsx, check and alter in FB_interest_id.xlsx
# also add the ones that cannot be found automatically
# Available and correct ones are saved as FB_interest_id.xlsx

In [ ]:
# manually check special cases
t = watcher.get_interests_given_query("Zenni")
t

No longer available:

Toys "R" Us, Jared, Hanes, Dockers, USA Today, The New York Times, The Onion, Fox News, CNN, CBS News, 60 Minutes, Central Market, the Hill, 48 Hours, American Housewife, Art Van Furniture, BabiesRUs, Century 21, Daily Caller, 
Flywheel Sports, Free Speech TV, Fusion News, Gordmans, MSNBC, NBC News, NCAA Women's Basketball, Natinal Football Leauge, Newsmax, Pier 1 Imports, The Wall Street Journal, The Washington Post, Warby Parker, Zenni Optical, world net daily

Among them closed/bankrupt:
Century 21 Stores, Flywheel Sports, Gordmans, Pier 1 Imports

In [8]:
FB_interest_id_raw = pd.read_excel("FB_interest_id_raw.xlsx")
# get the ones that need maunal input
set(brands) - set(FB_interest_id_raw['brand']) 

## create json files for data collection
Age, gender, location can be easily write manually, only interest id list are created with code.

In [ ]:
FB_interest_id = pd.read_excel("FB_interest_id.xlsx")

with open ("interest_list.json", "a") as file:
    for index, row in FB_interest_id.iterrows():
        json.dump({"or": [row['id']], "name": row['brand']}, file)
        file.write(',\n')

We are not sharing the interest ids of the brands (the content of interest_list.json), as it is not clear from Facebook whether we have the rights to share them. But you should be able to get them with the code provided so far. Then, use the json files as intructed by pysocialwatcher https://github.com/joaopalotti/pySocialWatcher to collect the data. The following cell provides the example json contents to be saved in the correponding json files.

## collect data 


In [12]:
to_collect_list = ["collecting_urban.json", "collecting_gender.json", "collecting_age.json"]
collected_files = ["collected_urban.csv", "collected_gender.csv", "collected_age.csv"]

for i in range(len(to_collect_list)):
    
    collecting = to_collect_list[i]
    collected = collected_files[i]
    
    print("start", collecting, "---")
    watcher = watcherAPI(api_version="15.0", outputname = collected)
    watcher.load_credentials_file("FB_credentials.csv")
    
    df = watcher.run_data_collection(collecting, remove_tmp_files = True)
    
    print(collected, "end---")

# Data preprocess

process the data collected and in the end get two files representing the upper and lower monthly active users. The files contains brands, and the upper/lower bound monthly active users in the different cateories.

In [14]:
def clean_pivot(csvfile, key_column):
    """
    input the file name of the csv file and the column name of the variable(age, gender, etc.)
    return the cleaned and pivoted version of dataframe for upper and lower mau
    """
    
    df = pd.read_csv(csvfile, index_col = 0)
    df = df[['interests', key_column, 'mau_audience_upper_bound', 'mau_audience_lower_bound']]
    df = df.assign(id = df['interests'].str.extract("\[([0-9]+)\]")) 
    
    upper = df[['id', 'mau_audience_upper_bound', key_column]]
    upper = upper.pivot(index = 'id', columns = key_column, values = 'mau_audience_upper_bound')
    upper.columns.name = None
    
    lower = df[['id', 'mau_audience_lower_bound', key_column]]
    lower = lower.pivot(index = 'id', columns = key_column, values = 'mau_audience_lower_bound')
    lower.columns.name = None
    
    return upper, lower

In [15]:
# process different categories individually
upper_geo, lower_geo = clean_pivot("collected_urban.csv", 'geo_locations')
upper_geo.columns = ['all_US', 'urban']
lower_geo.columns = ['all_US', 'urban']

upper_gender, lower_gender = clean_pivot("collected_gender.csv", 'genders')
upper_gender.columns = ['male', 'female']
lower_gender.columns = ['male', 'female']

upper_age, lower_age = clean_pivot("collected_age.csv", 'ages_ranges')
upper_age.columns = ['age18_24', 'age25_34', 'age35_44', 'age45_54', 'age55_64', 'age65plus']
lower_age.columns = ['age18_24', 'age25_34', 'age35_44', 'age45_54', 'age55_64', 'age65plus']

In [16]:
# put together
upper_mau = pd.concat([upper_geo, upper_age, upper_gender], axis = 1)
lower_mau = pd.concat([lower_geo, lower_age, lower_gender], axis = 1)

FB_interet_id = pd.read_excel("FB_interest_id.xlsx", dtype = {'id': str})[['brand', 'id']]
upper_mau = pd.merge(FB_interet_id, upper_mau, on = 'id').drop(columns = ['id'])
lower_mau = pd.merge(FB_interet_id, lower_mau, on = 'id').drop(columns = ['id'])

In [17]:
# find not valid case where the upper and lower bound for all US are both 1000
not_valid = pd.merge(upper_mau[['brand', 'all_US']], lower_mau[['brand', 'all_US']], on = 'brand')
not_valid = not_valid.set_index('brand')
not_valid = not_valid == 1000
not_valid = list(not_valid[not_valid.sum(axis = 1) == 2].index.values)

# delete
upper_mau = upper_mau[~upper_mau['brand'].isin(not_valid)]
lower_mau = lower_mau[~lower_mau['brand'].isin(not_valid)]

# save
upper_mau.to_csv('upper_mau.csv', index = False)
lower_mau.to_csv('lower_mau.csv', index = False)